<a href="https://colab.research.google.com/github/Keerthu8999/GoogleColab/blob/main/Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

In [1]:
import logging

import pandas as pd
from simpletransformers.ner import NERModel, NERArgs

In [2]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
import pandas as pd
data = pd.read_csv("/content/sample_data/ner_dataset.csv",encoding="latin1" )
data =data.fillna(method ="ffill")
sentence_numbers = [int(sentence.split(": ")[1]) - 1 for sentence in data["Sentence #"]]
data.insert(1, "sentence_id", sentence_numbers, True)
print(sentence_numbers[:30])
data.head(30)


In [ ]:
data.rename(columns={"Word":"words","Tag":"labels"}, inplace =True)
data.head(30)

In [ ]:
data = data.drop(columns=['Sentence #', 'POS'])


In [54]:
data.head(30)
print("B-Geo".upper())


B-GEO


In [59]:
data['labels'] = data['labels'].str.upper()
data.head(30)


In [63]:
eval_data1 = data.head(53)

In [ ]:
eval_data1

In [ ]:
train_data = [
    [0, "Harry", "B-PER"],
    [0, "Potter", "I-PER"],
    [0, "was", "O"],
    [0, "a", "O"],
    [0, "student", "B-MISC"],
    [0, "at", "O"],
    [0, "Hogwarts", "B-LOC"],
    [1, "Albus", "B-PER"],
    [1, "Dumbledore", "I-PER"],
    [1, "founded", "O"],
    [1, "the", "O"],
    [1, "Order", "B-ORG"],
    [1, "of", "I-ORG"],
    [1, "the", "I-ORG"],
    [1, "Phoenix", "I-ORG"],
]
train_data = pd.DataFrame(
    train_data, columns=["sentence_id", "words", "labels"]
)
train_data.head(5)

In [ ]:
data = pd.DataFrame(
    data, columns=["sentence_id", "words", "labels"]
)
data.head(5)

In [72]:
print(type(data.labels))
print(type(train_data.labels))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [78]:
unique_labels = data['labels'].unique()
print(unique_labels)

['O' 'B-GEO' 'B-GPE' 'B-PER' 'I-GEO' 'B-ORG' 'I-ORG' 'B-TIM' 'B-ART'
 'I-ART' 'I-PER' 'I-GPE' 'I-TIM' 'B-NAT' 'B-EVE' 'I-EVE' 'I-NAT']


In [31]:
eval_data = [
    [0, "Sirius", "B-PER"],
    [0, "Black", "I-PER"],
    [0, "was", "O"],
    [0, "a", "O"],
    [0, "prisoner", "B-MISC"],
    [0, "at", "O"],
    [0, "Azkaban", "B-LOC"],
    [1, "Lord", "B-PER"],
    [1, "Voldemort", "I-PER"],
    [1, "founded", "O"],
    [1, "the", "O"],
    [1, "Death", "B-ORG"],
    [1, "Eaters", "I-ORG"],
]
eval_data = pd.DataFrame(
    eval_data, columns=["sentence_id", "words", "labels"]
)

In [82]:
# Configure the model
model_args = NERArgs(overwrite_output_dir=True)
model_args.labels_list = ['O', 'B-GEO', 'B-GPE', 'B-PER', 'I-GEO', 'B-ORG', 'I-ORG', 'B-TIM', 'B-ART',
 'I-ART', 'I-PER', 'I-GPE', 'I-TIM', 'B-NAT', 'B-EVE', 'I-EVE', 'I-NAT']
model_args.train_batch_size = 16
model_args.evaluate_during_training = True

model = NERModel(
    "roberta", "roberta-base", args=model_args, use_cuda=False

)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.train_model(data, eval_data=eval_data1)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
result, model_outputs, preds_list = model.eval_model(eval_data)

In [10]:
predictions, raw_outputs = model.predict(["Hermione was the best in her class"])
print(predictions)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Hermione': 'I-LOC'}, {'was': 'B-PER'}, {'the': 'B-PER'}, {'best': 'O'}, {'in': 'O'}, {'her': 'O'}, {'class': 'I-LOC'}]]
